# Chrome Dino re-write 2023

Use Pytorch, stable-baselines3, etc. to implement an agent to play chrome://dino with reinforcement learning.

## dependencies
- CUDA-enabled Pytorch (Pytorch 2.00; CUDA 1.18)
- Stable-Baselines3 (with extras like OpenCV): https://stable-baselines3.readthedocs.io/en/master/
- Protobuf (a training dependency) (3.20.*)
- pytesseract (interface to Google Tesseract)
- Google Tesseract-OCR ((5.3.1.20230401)
- Gymn (gym v0.21 since this is used by Stable-Baselines3; RL environment library): https://gymnasium.farama.org/

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
                                              0.0/2.6 GB ? eta -:--:--
                                              0.0/2.6 GB ? eta -:--:--
                                              0.0/2.6 GB 2.1 MB/s eta 0:21:00
                                              0.0/2.6 GB 3.1 MB/s eta 0:13:59
                                              0.0/2.6 GB 4.5 MB/s eta 0:09:45
                                              0.0/2.6 GB 6.0 MB/s eta 0:07:17
                                              0.0/2.6 GB 9.0 MB/s eta 0:04:51
                                              0.0/2.6 GB 10.7 MB/s eta 0:04:05
                                              0.0/2.6 GB 13.3 MB/s eta 0:03:17
                                              0.0/2.6 GB 14.1 MB/s eta 0:03:06
                                              0.0/2.6 GB 13.9 MB/s eta 0:03:08
                                              0.0/2.6 GB 15.7 MB/s eta 0:02:47
              

In [10]:
# needed to support old version of gym (0.21) used with Stable-Baselines3
# gym 0.21 has installation issue; gym moved to Gymnasium; SB3 still does not support Gymnasium 
!pip install setuptools==66 Cmake git+https://github.com/openai/gym.git@9180d12e1b66e7e2a1a622614f787a6ec147ac40

  Cloning https://github.com/openai/gym.git (to revision 9180d12e1b66e7e2a1a622614f787a6ec147ac40) to c:\users\jakce\appdata\local\temp\pip-req-build-lgkk0owa
  Resolved https://github.com/openai/gym.git to commit 9180d12e1b66e7e2a1a622614f787a6ec147ac40
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1625702 sha256=e95587207bc5519129cb1c2113abb947eaff9cd430ef526c72efe36f1ef85299
  Stored in directory: c:\users\jakce\appdata\local\pip\cache\wheels\f0\ee\c0\6742dc49f0af8513cbebcc9e5bccce66351fad2f8e093e3301
Successfully built gym


  Running command git clone --filter=blob:none --quiet https://github.com/openai/gym.git 'C:\Users\jakce\AppData\Local\Temp\pip-req-build-lgkk0owa'
  Running command git rev-parse -q --verify 'sha^9180d12e1b66e7e2a1a622614f787a6ec147ac40'
  Running command git fetch -q https://github.com/openai/gym.git 9180d12e1b66e7e2a1a622614f787a6ec147ac40
  Running command git checkout -q 9180d12e1b66e7e2a1a622614f787a6ec147ac40


In [11]:
!pip install stable-baselines3[extra] protobuf==3.20.*

  Using cached stable_baselines3-1.8.0-py3-none-any.whl (174 kB)
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
  Using cached pandas-2.0.0-cp310-cp310-win_amd64.whl (11.2 MB)
  Using cached matplotlib-3.7.1-cp310-cp310-win_amd64.whl (7.6 MB)
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached tensorboard-2.12.2-py3-none-any.whl (5.6 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached rich-13.3.4-py3-none-any.whl (238 kB)
  Using cached ale_py-0.7.4-cp310-cp310-win_amd64.whl (904 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached AutoROM.accept_rom_license-0.6.1-py3-none-any.whl
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached grpcio-1.53.0-cp310-cp310-win_amd64.whl (3.9 MB)
  Using cached google_auth-2.17.3-py2.py3-none-any.whl (178 kB)
  Using cached go

In [12]:
!pip install mss pydirectinput pytesseract

In [14]:
from gym import Env
from gym.spaces import Box, Discrete

## set up environment

In [ ]:
# create a base Gym environment for managing state
# Env must implement step and reset; optionally, render 
# https://www.gymlibrary.dev/api/core/
class ChromeDinoAgent(Env):
    
    # initialize environment shapes
    def __init__(self):
        super().__init__()
        
        # set up observations and actions
        self.observation_space = 
        self.action_space = 
        
        
    # run one timestep in the environment
    def step(self, action):
        
        
    # reset to initial state, return initial observation
    def reset(self):
        
        
    # compute render frames for visualization; render with cv2
    def render(self):
        
        
    # do an observation
    def observe_environment(self):
        
        
    # identify game over
    def game_over(self):
        

## train model

Train the DQN and save it using a Stable-Baselines3 callback

In [ ]:
# training imports
from stable_baselines3 import DQN
from stablebaselines3.common.callbacks import BaseCallback
import os